# Hello world
## Optimization algorithms: demo

<em><b>Note</b>: This is just a demo of optimization techniques - not related to any real-life problem.</em>

Imagine the world of strings where the environment demands that the strings are very fit. An unfit string cannot survive. 

How do we define the fitness of a string? The environment demands that the string should be as similar as possible to the target string $s$=''Hello, world''. The closer the string is to ''Hello, world'', the better its chances of survival.

<b> The problem:</b> given a set of random strings of length len($s$) over alphabet $\sigma=\{32 \ldots 122\}$, produce the string which best matches the environment, i.e. with a minimum fitness score. 

Let <em>Weighted Hamming Distance</em> between two strings $x$ and $y$ (both of length $n$) be defined as:<br>
$WH(x,y) = \sum_{i=0}^{n} abs(x[i] - y[i])$

$WH(x,y)$ estimates how far is $x$ from $y$. The lower $WH(x,y)$, the closer string $x$ approaches the target string $y$.<br>
In the space of $\sigma^n$ of all possible strings, we are looking for a global minimum - the string $m$ such that $WH(m,s)$ is minimized. <br>

If we compute $WH(x,s)$ for an arbitrary string $x$, we say that we evaluate the <em>fitness</em> of $x$ with respect to the target string $s$. This is our <em>fitness function</em>. 

## 1. Preparation

In [1]:
# flag which can be turned on/off to print the steps of each optimization
print_steps = True

### 1.1. String fitness function 

In [2]:
s = 'Hello, world'
n = len(s)

# fitness function - weighted hamming distance
string_fitness = lambda x: sum([abs(ord(x[i])- ord(s[i])) for i in range(n)])

### 1.2. Generating initial random strings

In [5]:
import random

alphabet = range(32,123)

def get_random_string(n, sigma):    
    t = [chr(random.choice(sigma)) for i in range(n)]
    return ''.join(t)

# build initial population of many random strings of target length n
def get_rand_population(population_size, n, sigma):
    population=[]
    for i in range(0,population_size):
        individual_str = get_random_string(n, sigma)
        population.append(individual_str)
    return population

# test
print(get_random_string(len(s), alphabet))
print(get_rand_population(3, len(s), alphabet ))

YGT<`Z_m.E6@
['^>SjW0S2h\\wH', 'yCr=pMO%3r]E', 'ACbDShwul,Dq']


## 2. Random optimize

The simplest possible idea is to try a vaste amount of random solutions and select the one with the best fitness score. Let's see how close we can get to the target $s$=''Hello, world'' with this approach.

After all, isn't evolution just a lottery? ''Physics makes rules, evolution rolls the dice'' ( Charles Cockell. "The Equations of Life").

In [17]:
def random_optimize(population, fitness_function):
    best_score = None
    best_solution = None
    for i in range(len(population)):
        # Calculate fitness 
        score = fitness_function(population[i])

        # Compare it to the best one so far
        if best_score == None or score < best_score:
            best_score = score
            best_solution = population[i]
            if print_steps:
                print(best_solution, " -> score:", best_score)
            
    return (best_score, best_solution, len(population))


# Now the test
string_population = get_rand_population(204800, len(s), alphabet)

best_score, best_sol, iterations = random_optimize (string_population, string_fitness)

`zww&1IOt%RR  -> score: 352
0iNx4(h^CsEd  -> score: 314
yN4cS67r`Yxf  -> score: 257
^YR[hN*fbGVS  -> score: 240
orqu;1avmfvz  -> score: 235
CJLWs!"zez,i  -> score: 192
&[Ujt# :oVp[  -> score: 185
4^q(p#%paes]  -> score: 163
DGtwi/NqZqto  -> score: 155
mhyTx#(kgzab  -> score: 144
K\hZl,5isCfi  -> score: 134
:1glo<.qeslh  -> score: 122
<djZd$,zisRY  -> score: 111
AW]op)#fgo[R  -> score: 109
AhkYe$#rsyfK  -> score: 98


In [18]:
print()
print("*************Rand optimize**************")
print("trials:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Rand optimize**************
trials:204800, best solution:'AhkYe$#rsyfK', best score:98


Randomly trying different solutions is very inefficient because the probability of getting anything close to perfect Hello, world is $(\frac{1}{90})^{12}$. No matter how many random guesses you try – the fitness of the resulting string is very low (the distance from the target remains high).

## 3. Hill climbing

We did not advance far with the random optimization. This happens because each time we come up with some solution and evaluate its fitness - we discard it and try another one - not related to the previous.

The idea behing the <em>hill climbing</em> optimization is to start at a random point in space (choose one random string), and try to move from this point into a direction of better fitness. We will generate a set of neighbors for a randomly selected string, and see if any of these neighbors improve the overall fitness score of the string. 

We continue moving into the direction of this better solution, until there is no more improvement. 

<img src="images/hillclimbing.jpg" style="height:200px;">


The algorithm may produce an optimal solution, but it may also get stuck at the local minimum.
<img src="images/localmin.jpg" style="height:200px;">

In [19]:
def hillclimb_optimize(start_point, fitness_function):
    sol = start_point
    score = fitness_function(sol)
    iterations = 0
    
    # Main loop
    while 1:
        iterations += 1
        
        current_best_score = score
        # Create  neighboring solutions
        neighbors=[]

        for j in range(len(s)):
            # Change character at position j one up or one down
            if ord(sol[j])>32:
                neighbors.append(sol[0:j]+chr(ord(sol[j])-1)+sol[j+1:])
            if ord(sol[j])<122:
                neighbors.append(sol[0:j]+chr(ord(sol[j])+1)+sol[j+1:])
        
        for n_str in neighbors:
            n_score = fitness_function(n_str)
            if n_score < score:
                score = n_score
                sol = n_str
                if print_steps:
                    print(sol, " -> score:", score)      
        
        # If there's no improvement, then we've reached the bottom of the hill
        if score == current_best_score:
            break        
        
        if score == 0:
            break        
        
    return (score, sol, iterations)

# Now the test

rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = hillclimb_optimize(rand_str, string_fitness)

<5 4 ZKG[K5r  -> score: 536
=5 4 ZKG[K5r  -> score: 535
>5 4 ZKG[K5r  -> score: 534
?5 4 ZKG[K5r  -> score: 533
@5 4 ZKG[K5r  -> score: 532
A5 4 ZKG[K5r  -> score: 531
B5 4 ZKG[K5r  -> score: 530
C5 4 ZKG[K5r  -> score: 529
D5 4 ZKG[K5r  -> score: 528
E5 4 ZKG[K5r  -> score: 527
F5 4 ZKG[K5r  -> score: 526
G5 4 ZKG[K5r  -> score: 525
H5 4 ZKG[K5r  -> score: 524
H6 4 ZKG[K5r  -> score: 523
H7 4 ZKG[K5r  -> score: 522
H8 4 ZKG[K5r  -> score: 521
H9 4 ZKG[K5r  -> score: 520
H: 4 ZKG[K5r  -> score: 519
H; 4 ZKG[K5r  -> score: 518
H< 4 ZKG[K5r  -> score: 517
H= 4 ZKG[K5r  -> score: 516
H> 4 ZKG[K5r  -> score: 515
H? 4 ZKG[K5r  -> score: 514
H@ 4 ZKG[K5r  -> score: 513
HA 4 ZKG[K5r  -> score: 512
HB 4 ZKG[K5r  -> score: 511
HC 4 ZKG[K5r  -> score: 510
HD 4 ZKG[K5r  -> score: 509
HE 4 ZKG[K5r  -> score: 508
HF 4 ZKG[K5r  -> score: 507
HG 4 ZKG[K5r  -> score: 506
HH 4 ZKG[K5r  -> score: 505
HI 4 ZKG[K5r  -> score: 504
HJ 4 ZKG[K5r  -> score: 503
HK 4 ZKG[K5r  -> score: 502
HL 4 ZKG[K5r  -> sco

In [20]:
print()
print("*************Hill climbing****************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Hill climbing****************
steps:537, best solution:'Hello, world', best score:0


## 3. Simulated annealing
The idea of the <em>simulated annealing</em> is borrowed from physics. In metalurgical annealing we heat a metal (alloy) to a very high temperature, the crystal bonds break, and the atoms diffuse more freely. If we cool it slowly, the atoms tend to form more regular crystals producing an alloy with the low termodynamics energy.  

<img style="height:250px;float:right;padding:4px;" src="images/sim_annealing.jpg" >

In the <em>simulated annealing</em> algorithm we set the initial temperature very high, and then we generate a single random neighbor of the current solution. The fitness of this neihgbor can be better or worse that that of the current solution. When the temperature is high, the probability of selecting worse solution is higher. This allows to better explore the search space and get out of the local minimum. The temperature gradually decreases, and so at the end we do not accept worse solutions.

The criterion of accepting ''bad'' solutions:

$p=e^{\frac{-\Delta F}{T}}>R(0,1)$

where $T$ is the current temperature, $R(0,1)$ is a random number between $0$ and $1$, and $\Delta F$ is the difference between the fitness score of new solution and the old solution.

Since the temperature $T$ (the willingness to accept a worse solution) starts very high,
the exponent will be close to 0, and $p$ will almost be 1. As the temperature decreases, the difference between the new fitness score and the old one becomes more important - a bigger difference leads to a lower probability, so the
algorithm will not accept solutions which do not improve fitness - converging to a local minimum after exploring a large global search space.

In [21]:
import math

def annealing_optimize(start_sol, fitness_function, T=10000.0, cool=0.95, step=1):    
    sol = start_sol
    iterations = 0
    
    # Graduate cooling
    while T > 0.01: 
        score = fitness_function(sol)
        
        # Choose one of spots randomly
        i = random.randint(0, len(sol) - 1)

        # Choose rand direction to change the character at position i
        dir = random.randint(-step, step)

        change = ord(sol[i]) + dir
        # out of domain
        if change > 122 or change < 32:
            continue
    
        iterations += 1
        
        # Create a new solution with one of the characters changed
        new_sol = sol[:i] + chr(change) + sol[i+1:]

        # Calculate the new cost
        new_score = fitness_function(new_sol)        

        # Does it make the probability cutoff?
        p = pow(math.e, -(new_score - score) / T)
        
        if new_score < score or p > random.random() :
            sol = new_sol
            score = new_score
            if print_steps:
                print(sol, "-> score:", score)       
        
        if score == 0:
            break
            
        # Decrease the temperature
        T = T * cool
    
    return (score, sol, iterations)

# now test
rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = annealing_optimize(rand_str, string_fitness,
                                          T=204800.0, cool=0.999, step=1)

g-?Fje*D w(N -> score: 467
g-?Fje*D w)N -> score: 466
g-?Fje*D w)N -> score: 466
g->Fje*D w)N -> score: 467
g->Fje*D w)N -> score: 467
g->Fje*D w)O -> score: 466
g-=Fje*D w)O -> score: 467
g-=Fje*D w)O -> score: 467
g-=Fje*D!w)O -> score: 466
g-=Fje*D!w)O -> score: 466
g-=Fke*D!w)O -> score: 465
g-=Fke*D!w*O -> score: 464
g-=Fke*D!w*P -> score: 463
g-=Fke)D!w*P -> score: 462
g-=Fke)D!w*P -> score: 462
g-=Fke)D!w*P -> score: 462
g-=Fke)D!v*P -> score: 461
g-=Fkd)D!v*P -> score: 460
g-=Fkd)D!v*P -> score: 460
g-=Fjd)D!v*P -> score: 461
g-=Fjd)D!v*P -> score: 461
g-=Gjd)D!v*P -> score: 460
g-=Gjd(D!v*P -> score: 459
g-=Gjd(D!v+P -> score: 458
g-=Gjd(D!v+Q -> score: 457
g-=Gjd(D!v+Q -> score: 457
g-=Gje(D!v+Q -> score: 458
g->Gje(D!v+Q -> score: 457
g,>Gje(D!v+Q -> score: 458
g,>Gje'D!v+Q -> score: 457
g,>Gje'E!v+Q -> score: 456
g,>Gje'E!v+Q -> score: 456
h,>Gje'E!v+Q -> score: 457
h,>Gje'E!w+Q -> score: 458
h,>Gje'D!w+Q -> score: 459
h,>Gje'D!w+Q -> score: 459
h,>Gje'E!w+Q -> score: 458
i

In [22]:
print()
print("*************Simulated annealing***************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Simulated annealing***************
steps:13604, best solution:'Hello, world', best score:0


## 4. Genetic algorithm

This optimization technique is inspired by the theory of evolution. The algorithm starts with a population of random individuals, and selects the ones with the best fitness score (elite). It continues to the next generation with this group. In order to enrich the genetic pool in the current generation, the algorithm adds random mutations and crossover to the elite group. After predefined number of generations, the algorithm returns the top-fit individual. 

### 4.1. Mutation and crossover

In [23]:
# Mutation Operation
def string_mutation(individual):
    i = random.randint(0, len(individual) - 1)

    # mutation changes character at random position to any valid character   
    rchar = chr(random.randint(32, 122))

    individual = individual[0:i] + rchar + individual[(i + 1):]
    return individual


# Mate operation (crossover)
def string_crossover (s1, s2):
    # find the point of crossover
    i = random.randint (0, len(s1)-1)
    return s1[:i]+s2[i:]

### 4.2. Algorithm

Initial population - a list of random strings

In [24]:
def genetic_optimize(population, fitness_function,
                     mutation_function, mate_function,
                     mutation_probability, elite_ratio,
                     maxiterations):

    # How many winners to consider from each generation?
    original_population_size = len(population)
    top_elite = int(elite_ratio * original_population_size)

    # Main loop
    iterations = 0
    for i in range(maxiterations):
        iterations += 1
        individual_scores = [(fitness_function(v), v) for v in population]
        individual_scores.sort()
        ranked_individuals = [v for (s, v) in individual_scores]

        # Start with the pure winners
        population = ranked_individuals[0:top_elite]

        # Add mutated and bred forms of the winners
        while len(population) < original_population_size:
            if random.random() < mutation_probability:
                # Mutation
                c = random.randint(0, top_elite)
                population.append(mutation_function(ranked_individuals[c]))
            else:
                # Crossover
                c1 = random.randint(0, top_elite)
                c2 = random.randint(0, top_elite)
                population.append(mate_function(ranked_individuals[c1], ranked_individuals[c2]))

        # Print current best score
        if print_steps:
            print(individual_scores[0][1]," -> score:", individual_scores[0][0])

        if individual_scores[0][0] == 0:
            return (individual_scores[0][0],individual_scores[0][1], iterations)

    # returns the best solution
    return (individual_scores[0][0], individual_scores[0][1], iterations)



string_population = get_rand_population(2048, len(s), alphabet)
best_score, best_sol, iterations  = genetic_optimize(string_population, string_fitness,
                                string_mutation, string_crossover,
                                mutation_probability=0.25, elite_ratio=0.1,
                                maxiterations=100)
print()
print("*****************GENETIC ALGORITHM ***************")
print("generations:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


aljMt/8iineW  -> score: 141
9ljMt/8iineW  -> score: 131
F\mqmB*tguzj  -> score: 85
IhnqqA uujlg  -> score: 53
Lenkp%!wljlg  -> score: 30
Leiup+ zopkb  -> score: 26
Lenlp+ zopkb  -> score: 16
Fenlp+ zopkb  -> score: 14
Lelkp+ wopkb  -> score: 12
Ienkp+ woplb  -> score: 10
Ielkp+ wopkb  -> score: 9
Iellp+ woplb  -> score: 7
Iellp+ woslb  -> score: 6
Iellp+ wopld  -> score: 5
Iellp+ wopld  -> score: 5
Iellp+ worle  -> score: 4
Iello+ wosld  -> score: 3
Iello+ world  -> score: 2
Gello+ world  -> score: 2
Hello+ world  -> score: 1
Hello, world  -> score: 0

*****************GENETIC ALGORITHM ***************
generations:21, best solution:'Hello, world', best score:0


### This is the end of the ''Hello, world'' demo.
Copyright &copy; 2022 Marina Barsky